In [2]:
%matplotlib inline

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from skimage import transform as tf
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np

import cv2

from LFWDataset import LFWDataset
from SiameseNet import SiameseNet
from ContrastiveLoss import ContrastiveLoss
from augmentation import *
# from p1a import data_aug

In [3]:
%matplotlib inline
def show(img, filename=None, save=False):
    npimg = img.numpy()
    plt.axis("off")
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    plt.show()
    
    if save and filename is not None:
        plt.savefig(filename)

def show_plot(iteration,loss, filename='loss.png', save=False):
    plt.plot(iteration,loss)
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()
    if save:
        plt.savefig(filename)

In [4]:
trainset = LFWDataset(train=True,
                      transform=transforms.Compose([augmentation, transforms.Scale((128,128)),
                                                                      transforms.ToTensor()
                                                                      ]))
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)




In [5]:
# vis_dataloader = DataLoader(trainset,
#                         shuffle=True,
#                         num_workers=2,
#                         batch_size=8)
# dataiter = iter(vis_dataloader)

# example_batch = next(dataiter)
# concatenated = torch.cat((example_batch[0],example_batch[1]),0)

# show(torchvision.utils.make_grid(concatenated))
# print(example_batch[2].numpy())

In [11]:
net = SiameseNet(p1b=True).cuda()
# criterion = nn.BCELoss()
criterion = ContrastiveLoss()
learning_rate = 1e-6
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [12]:
print(net)

SiameseNet (
  (cnn): Sequential (
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU (inplace)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU (inplace)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (7): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (11): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  )
  (fc1): Sequential (
    (0): Linear (131072 -> 1024)
    (1): ReLU (inplace)
    (2): BatchNorm2d(1024, 

In [13]:
counter = []
loss_history = [] 
iteration_number= 0
epochs = 30

In [14]:
for epoch in range(epochs):
    for i, data in enumerate(trainloader,0):
        img0, img1 , label = data
        img0, img1 , label = Variable(img0).cuda(), Variable(img1).cuda() , Variable(label).cuda()
#         output = net(img0, img1)
        output1, output2 = net(img0,img1)
        label = label.unsqueeze(1).float()

        optimizer.zero_grad()
#         loss = criterion(output, label)
        loss = criterion(output1, output2, label)
        loss.backward()
        optimizer.step()
        if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss.data[0]))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss.data[0])

Epoch number 0
 Current loss 344.980133057



Process Process-4:
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self.run()
  File "/home/zdavidli/.local/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    samples = collate_fn([dataset[i] for i in batch_indices])
    self._target(*self._args, **self._kwargs)
  File "/home/zdavidli/.local/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
  File "/home/zdavidli/.local/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 109, in default_collate
    samples = collate_fn([dataset[i] for i in batc

KeyboardInterrupt: 

In [ ]:
# show_plot(counter,loss_history, save=True)
plt.figure(figsize=(10, 4))
plt.plot(counter,loss_history)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.savefig('loss.png')
plt.show()


In [ ]:
torch.save(net.state_dict(), "p1b")

In [ ]:
testset = LFWDataset(test=True,
                     transform=transforms.Compose([transforms.Scale((128, 128)),
                                                                      transforms.ToTensor()
                                                                      ]))
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=2)

In [ ]:
net.load_state_dict(torch.load('p1b'))

In [ ]:
right=wrong=0.

for i, data in enumerate(testloader,0):
    img0, img1, label = data    
    img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()
    
    output1, output2 = net(img0,img1)
    dist = F.pairwise_distance(output1, output2)
    print(dist)
    print(label)
    for x,y in zip(dist, label):
        if (x.data[0]>=11 and y.data[0]==1) or (x.data[0]<11 and y.data[0]==0):
            right+=1
        else:
            wrong+=1
    

In [ ]:
#     print(right, wrong)
testacc = right/(right+wrong)
print("Accuracy on test set: {:.2f}".format(testacc))

In [ ]:
right=wrong=0.

for i, data in enumerate(trainloader,0):
    img0, img1, label = data    
    img0, img1, label = Variable(img0).cuda(), Variable(img1).cuda(), Variable(label).cuda()
    
    output = net(img0,img1)
    for x,y in zip(dist, label):
        if (x.data[0]<=11 and y.data[0]==1) or (x.data[0]>11 and y.data[0]==0):
            right+=1
        else:
            wrong+=1
            

In [ ]:
#     print(right, wrong)
trainacc = right/(right+wrong)
print("Accuracy on train set: {:.2f}".format(trainacc))

In [ ]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.autograd.Variable(torch.randn(3), requires_grad=True)
target = torch.autograd.Variable(torch.FloatTensor(3).random_(2))
print(input, target)
output = loss(m(input), target)
output.backward()

In [ ]:
from PIL import Image
im = Image.open("lfw/Taufik_Hidayat/Taufik_Hidayat_0001.jpg")

plt.imshow(im)

In [ ]:
plt.imshow(data_aug(im))